In [3]:
"""
-first part:
seq2seq without teacher forcing
-second part:
seq2seq without teacher forcing, with others' future convLSTM
concat states with decoder LSTM and then predict
"""
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers import Lambda,Concatenate,Flatten,ConvLSTM2D
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import backend as K
from keras.models import load_model
import sys,glob,io,random
if '/Users/maojuntao/Downloads/project/' not in sys.path:
    sys.path.insert(0, '/Users/maojuntao/Downloads/project/')
from dataLayer import DataLayer
import cost as costfunc
from config import cfg
from dataIO import clip_xyz
from utility import reshape2second_stacks,get_data
from utility import get_shuffle_index,shuffle_data,get_gt_target_xyz,get_gt_target_xyz_oth
from utility import slice_layer
from random import shuffle
import matplotlib.pyplot as plt
import cPickle as pickle
import numpy as np
import pdb
### ====================Graph def====================
def onelayer_tar_seq2seq():
    # The first part is unchanged
    if not cfg.input_mean_var:
        encoder_inputs = Input(shape=(None, num_encoder_tokens))
    else:
        encoder_inputs = Input(shape=(None, num_decoder_tokens))    
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    states = [state_h, state_c]

    # Set up the decoder, which will only process one timestep at a time.
    decoder_inputs = Input(shape=(1, num_decoder_tokens))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_dense = Dense(num_decoder_tokens,activation='tanh')

    all_outputs = []
    inputs = decoder_inputs
    for _ in range(max_decoder_seq_length):
        # Run the decoder on one timestep
        decoder_states, state_h, state_c = decoder_lstm(inputs,
                                                 initial_state=states)
        outputs = decoder_dense(decoder_states)
        # Store the current prediction (we will concatenate all predictions later)
        all_outputs.append(outputs)
        # Reinject the outputs as inputs for the next loop iteration
        # as well as update the states
        inputs = outputs
        states = [state_h, state_c]

    # Concatenate all predictions
    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

    # Define and compile model as previously
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model


if __name__ == '__main__':
    #experiment = 1
    batch_size = 32  # Batch size for training.
    epochs = 200  # Number of epochs to train for.
    latent_dim = 64  # Latent dimensionality of the encoding space.

    fps = 30
    num_encoder_tokens = 3*fps
    num_decoder_tokens = 6
    max_encoder_seq_length = cfg.running_length
    max_decoder_seq_length = cfg.predict_step
    # num_user = 48


    model = onelayer_tar_seq2seq()
    model.compile(optimizer='Adam', loss='mean_squared_error')



    #### ====================data====================
    ## load data just as in Fov_seq2seq.py
    # if cfg.use_xyz:
    #     all_video_data = pickle.load(open('/scratch/cl2840/360video/data/new_exp_'+str(experiment)+'_xyz.p','rb'))
    #     data_dim = 3
    # all_video_data = clip_xyz(all_video_data)
    # datadb = all_video_data.copy()
    ##### data format 3or4--4
    # video_data_train = pickle.load(open('/scratch/cl2840/360video/data/shanghai_dataset_xyz_train.p','rb'))    
    #### format5
    video_data_train = pickle.load(open('/Users/maojuntao/Downloads/project/data 2/shanghai_dataset_xyz_train.p','rb')) 
    video_data_train = clip_xyz(video_data_train)
    datadb = video_data_train.copy()


    _video_db,_video_db_future,_video_db_future_input = get_data(datadb,pick_user=False)
    # total_num_samples = _video_db.shape[0]

    if cfg.shuffle_data:
        #shuffle the whole dataset
        # index_shuf = get_shuffle_index(total_num_samples)
        index_shuf = pickle.load(open('index_shuf'+'_exp'+str(experiment)+'.p','rb'))
        _video_db = shuffle_data(index_shuf,_video_db)
        _video_db_future = shuffle_data(index_shuf,_video_db_future)
        _video_db_future_input = shuffle_data(index_shuf,_video_db_future_input)


    # num_testing_sample = int(0.15*total_num_samples)#use last few as test
    num_testing_sample = 1
    if cfg.input_mean_var:
        encoder_input_data = get_gt_target_xyz(_video_db[:-num_testing_sample,:,:])
    else:
        encoder_input_data = _video_db[:-num_testing_sample,:,:]
    decoder_target_data = get_gt_target_xyz(_video_db_future)[:-num_testing_sample,:,:]
    # decoder_input_data = get_gt_target_xyz(_video_db_future_input)[:-num_testing_sample,-1,:][:,np.newaxis,:]
    decoder_input_data = get_gt_target_xyz(_video_db_future_input)[:-num_testing_sample,0,:][:,np.newaxis,:]



    ### ====================Training====================
    # model = load_model('fov_s2s_noteacherforcing_epoch21-0.0759.h5')
    # tag = 'notFor_tanh_newdata_exp2_epoch'
    # tag = 'fctar_seqseq_shanghai_traintest_split_predmeanvar_Aug9'
    # tag = 'fctar_seqseq_shanghai_traintest_split_meanvarmeanvar_Aug9'
    tag = 'fctar_seqseq_THU_predmeanvar_Sep5'
    model_checkpoint = ModelCheckpoint(tag+'{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss', save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                     patience=3, min_lr=1e-6)
    stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
              shuffle=True,initial_epoch=0,
              callbacks=[model_checkpoint, reduce_lr, stopping])



    ### ====================Testing====================
    ##### data format 3or4--4
    # video_data_test = pickle.load(open('/scratch/cl2840/360video/data/shanghai_dataset_xyz_test.p','rb'))
    ### data format 5
    video_data_test = pickle.load(open('/Users/maojuntao/Downloads/project/data 2/shanghai_dataset_xyz_test.p','rb'))

    video_data_test = clip_xyz(video_data_test)
    datadb = video_data_test.copy()   ##arrange the test data
    _video_db,_video_db_future,_video_db_future_input = get_data(datadb,pick_user=False)

    if cfg.input_mean_var:
        _video_db = get_gt_target_xyz(_video_db)

    # model = load_model('fov_s2s_noteacherforcing.h5')
    # model = load_model('fov_s2s_noteacherforcing_epoch74-0.0087.h5')
    # model = load_model('notFor_tanh_newdata_epoch18-0.0761.h5')
    # model = load_model('notFor_tanh_newdata_exp2_epoch200-0.0302.h5')
    def decode_sequence_fov(input_seq):
        # Encode the input as state vectors.
        last_location = input_seq[0,-1,:][np.newaxis,np.newaxis,:]
        if not cfg.input_mean_var:
            last_mu_var = get_gt_target_xyz(last_location)
        else:
            last_mu_var = last_location
        decoded_sentence = model.predict([input_seq,last_mu_var])
        return decoded_sentence


    gt_sentence_list = []
    decoded_sentence_list = []
    # for seq_index in range(total_num_samples-num_testing_sample,total_num_samples):
    for seq_index in range(_video_db.shape[0]):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = _video_db[seq_index: seq_index + 1,:,:]

        decoded_sentence = decode_sequence_fov(input_seq)
        decoded_sentence_list+=[decoded_sentence]
        gt_sentence = _video_db_future[seq_index: seq_index + 1,:,:]
        gt_sentence_list+=[gt_sentence]
        decoder_target = get_gt_target_xyz(gt_sentence)
        # print('-')
        # print('Decoded sentence - decoder_target:', np.squeeze(np.array(decoded_sentence))[:,:3]-np.squeeze(decoder_target)[:,:3])

    pickle.dump(decoded_sentence_list,open('decoded_sentence'+tag+'.p','wb'))
    pickle.dump(gt_sentence_list,open('gt_sentence_list'+tag+'.p','wb'))
    print('Testing finished!')









'''
##################################################################################################################

    ### ==============Below is concating with others' future branch convLSTM===========================
    ## utility layers
    flatten_layer = Flatten()
    expand_dim_layer = Lambda(lambda x: K.expand_dims(x,1))
    Concatenatelayer = Concatenate(axis=1)
    get_dim1_layer = Lambda(lambda x: x[:,0,:])


    ### ====================Graph def====================
    # The first part is unchanged
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    states = [state_h, state_c]


    ###======convLSTM on others' future======
    other_fut_lstm = ConvLSTM2D(filters=latent_dim, kernel_size=(num_user-1, 3),
                       input_shape=(1, num_user-1, fps, 3),
                       padding='same', return_sequences=True, return_state=True)
    others_fut_inputs = Input(shape=(max_decoder_seq_length,num_user-1,fps,3))
    # others_fut_inputs = Input(batch_shape=(batch_size,1,num_user-1,fps,3))
    flatten_conv_lstm_state_dense = Dense(latent_dim)

    fut_outputs_sqns, others_state_h, others_state_c = other_fut_lstm(others_fut_inputs)




    # Set up the decoder, which will only process one timestep at a time.
    decoder_inputs = Input(shape=(1, num_decoder_tokens))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    # decoder_inputs = Input(batch_shape=(batch_size, 1, num_decoder_tokens))
    # decoder_lstm = LSTM(latent_dim, return_sequences=False, return_state=True, stateful=True)
    decoder_dense = Dense(num_decoder_tokens,activation='tanh')


    ## concat states
    all_outputs = []
    inputs = decoder_inputs
    for time_ind in range(max_decoder_seq_length):
        # Run the decoder on one timestep
        decoder_states, state_h, state_c = decoder_lstm(inputs,initial_state=states)

        # ### caution: it seems keras convLSTM is by default stateful, don't have to feed back last hidden states.
        # ### is this true?
        # fut_outputs, others_state_h, others_state_c = other_fut_lstm(others_fut_inputs)
        # # fut_outputs, others_state_h, others_state_c = other_fut_lstm(others_fut_inputs,initial_state=others_states)#erros?!!!
        
        # fut_outputs = identity_layer(fut_outputs_sqns[:,time_ind,:,:,:])
        fut_outputs = slice_layer(1,time_ind,time_ind+1)(fut_outputs_sqns)
        convlstm_state = flatten_layer(fut_outputs)
        convlstm_state = flatten_conv_lstm_state_dense(convlstm_state)
        concat_state = Concatenatelayer([get_dim1_layer(decoder_states),convlstm_state])
        outputs = decoder_dense(concat_state)
        outputs = expand_dim_layer(outputs)
        all_outputs.append(outputs)

        inputs = outputs
        states = [state_h, state_c]

        # others_fut_inputs = others_fut_inputs #TODO feed gt others for next step
        # others_states = [others_state_h, others_state_c]


    # Concatenate all predictions
    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)


    # Define and compile model as previously
    model = Model([encoder_inputs, others_fut_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='Adam', loss='mean_squared_error')




    #### ====================data====================
    # load cached data
    _video_db_tar = pickle.load(open('./cache/_video_db_tar_exp'+str(experiment)+'.p','rb'))
    _video_db_future_tar = pickle.load(open('./cache/_video_db_future_tar_exp'+str(experiment)+'.p','rb'))
    _video_db_future_input_tar = pickle.load(open('./cache/_video_db_future_input_tar_exp'+str(experiment)+'.p','rb'))
    _video_db_oth = pickle.load(open('./cache/_video_db_oth_exp'+str(experiment)+'.p','rb'))
    _video_db_future_oth = pickle.load(open('./cache/_video_db_future_oth_exp'+str(experiment)+'.p','rb'))
    _video_db_future_input_oth = pickle.load(open('./cache/_video_db_future_input_oth_exp'+str(experiment)+'.p','rb'))




    def data_sanity_check(_video_db_tar,_video_db_future_tar,_video_db_future_input_tar):
        sample_ind = np.random.randint(0,_video_db_future_input_tar.shape[0])
        assert (_video_db_tar[sample_ind,:][-1,:]-_video_db_future_input_tar[sample_ind,:][0,:]).sum()==0
        print(np.abs(_video_db_tar[sample_ind,:][-1,:]-_video_db_future_tar[sample_ind,:][0,:]))
        
    def _reshape_others_data(_video_db_oth):
        ## to match Input shape: others_fut_inputs
        _video_db_oth = _video_db_oth.transpose((1,2,0,3))
        _video_db_oth = _video_db_oth.reshape((_video_db_oth.shape[0],_video_db_oth.shape[1],_video_db_oth.shape[2],
                        fps,3))
        return _video_db_oth

    _video_db_oth = _reshape_others_data(_video_db_oth)
    _video_db_future_oth = _reshape_others_data(_video_db_future_oth)
    # _video_db_future_input_oth = _reshape_others_data(_video_db_future_input_oth)
    total_num_samples = _video_db_tar.shape[0]

    #### shuffle the whole dataset
    # index_shuf = get_shuffle_index(total_num_samples)
    index_shuf = pickle.load(open('index_shuf'+'_exp'+str(experiment)+'.p','rb'))
    print('Shuffle data before training and testing.')
    _video_db_tar = shuffle_data(index_shuf,_video_db_tar)
    _video_db_future_tar = shuffle_data(index_shuf,_video_db_future_tar)
    _video_db_future_input_tar = shuffle_data(index_shuf,_video_db_future_input_tar)

    # _video_db_oth = shuffle_data(index_shuf,_video_db_oth)
    _video_db_future_oth = shuffle_data(index_shuf,_video_db_future_oth)
    # _video_db_future_input_oth = shuffle_data(index_shuf,_video_db_future_input_oth)


    #### prepare training data
    data_sanity_check(_video_db_tar,_video_db_future_tar,_video_db_future_input_tar)
    num_testing_sample = int(0.15*total_num_samples)#use last few as test
    encoder_input_data = _video_db_tar[:-num_testing_sample,:,:]
    decoder_target_data = get_gt_target_xyz(_video_db_future_tar)[:-num_testing_sample,:,:]
    # decoder_input_data = get_gt_target_xyz(_video_db_future_input_tar)[:-num_testing_sample,-1,:][:,np.newaxis,:]
    decoder_input_data = get_gt_target_xyz(_video_db_future_input_tar)[:-num_testing_sample,0,:][:,np.newaxis,:]

    others_fut_input_data = _video_db_future_oth[:-num_testing_sample,:]





    ### ====================Training====================
    # model = load_model('fov_s2s_notfor_concatConvLSTM_epoch82-0.0168.h5')
    # model = load_model('fov_s2s_noTfor_concatConvLSTM_48user_shuffle_epoch10-0.0487.h5')
    # model = load_model('fov_s2s_noTfor_concatConvLSTM_48user_shuffle_epoch06-0.0155.h5')
    # model = load_model('fov_s2s_noTfor_concatConvLSTM_48user_shuffle_epoch26-0.0096.h5')
    model_checkpoint = ModelCheckpoint('concat_future_ConvLSTM_newdata_epoch{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss', save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                     patience=3, min_lr=1e-6)
    stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
    # Train model as previously 
    model.fit([encoder_input_data, others_fut_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,
              shuffle=True, initial_epoch=0,
              callbacks=[model_checkpoint, reduce_lr, stopping])






    ### ====================Testing====================
    # model = load_model('fov_s2s_notfor_concatConvLSTM_epoch82-0.0168.h5')
    # model = load_model('concat_future_ConvLSTM_newdata_epoch08-0.8951.h5')
    def decode_sequence_fov(input_seq,others_fut_input_seq):
        # Encode the input as state vectors.
        last_location = input_seq[0,-1,:][np.newaxis,np.newaxis,:]
        last_mu_var = get_gt_target_xyz(last_location)
        decoded_sentence = model.predict([input_seq,others_fut_input_seq,last_mu_var])
        return decoded_sentence

    gt_sentence_list = []
    decoded_sentence_list = []
    for seq_index in range(total_num_samples-num_testing_sample,total_num_samples):
        input_seq = _video_db_tar[seq_index: seq_index + 1,:,:]
        others_fut_input_seq = _video_db_future_oth[seq_index: seq_index + 1,:]
        decoded_sentence = decode_sequence_fov(input_seq,others_fut_input_seq)
        decoded_sentence_list+=[decoded_sentence]
        gt_sentence = _video_db_future_tar[seq_index: seq_index + 1,:,:]
        gt_sentence_list+=[gt_sentence]
        # print('-')
        # decoder_target = get_gt_target_xyz(gt_sentence)
        # print('Decoded sentence - decoder_target:', np.squeeze(np.array(decoded_sentence))[:,:3]-np.squeeze(decoder_target)[:,:3])

    pickle.dump(decoded_sentence_list,open('decoded_sentence.p','wb'))
    pickle.dump(gt_sentence_list,open('gt_sentence_list.p','wb'))
    print('Testing finished!')











    ####TimeDistributed

    # fut_outputs, others_state_h, others_state_c = other_fut_lstm(others_fut_inputs)
    # # convlstm_state = flatten_conv_lstm_state_dense(K.reshape(fut_outputs,(-1,max_decoder_seq_length,(num_user-1)*fps*3)))

    # td_layer = TimeDistributed(flatten_conv_lstm_state_dense)
    # # convlstm_state = td_layer(K.reshape(fut_outputs,(-1,max_decoder_seq_length,(num_user-1)*fps*3)))
    # convlstm_state = flatten_layer(fut_outputs)



    ####!!check compatibility, EVERY layer needs to be keras tensor (especially concat, slice, backend ops)

    # model1 = Model(encoder_inputs, encoder_outputs)

    # model2 = Model(others_fut_inputs, convlstm_state)
    # model5 = Model([encoder_inputs,others_fut_inputs,decoder_inputs], concat_state)
    # model5 = Model([encoder_inputs,others_fut_inputs,decoder_inputs], outputs)
    # model5 = Model([encoder_inputs,others_fut_inputs,decoder_inputs], decoder_outputs)


    # model3 = Model([decoder_inputs,encoder_inputs], decoder_states)
    # model3 = Model([decoder_inputs,encoder_inputs], state_h)




'''

Using TensorFlow backend.


('_video_ind = ', '098', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '099', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '214', 'per_video_db.shape = ', (28, 10, 90))
('_video_ind = ', '215', 'per_video_db.shape = ', (58, 10, 90))
('_video_ind = ', '212', 'per_video_db.shape = ', (84, 10, 90))
('_video_ind = ', '213', 'per_video_db.shape = ', (29, 10, 90))
('_video_ind = ', '210', 'per_video_db.shape = ', (28, 10, 90))
('_video_ind = ', '211', 'per_video_db.shape = ', (29, 10, 90))
('_video_ind = ', '090', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '092', 'per_video_db.shape = ', (102, 10, 90))
('_video_ind = ', '093', 'per_video_db.shape = ', (68, 10, 90))
video 094 only has 17 seconds. skip...
('_video_ind = ', '095', 'per_video_db.shape = ', (68, 10, 90))
('_video_ind = ', '096', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '097', 'per_video_db.shape = ', (136, 10, 90))
('_video_ind = ', '133', 'per_video_db.shape = ', (34, 10, 90))

/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_4:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was 

Epoch 2/200
9100/9100 [==============================] - 7s 813us/step - loss: 0.1069 - val_loss: 0.1208
Epoch 3/200
 160/9100 [..............................] - ETA: 7s - loss: 0.1075

/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_4:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was 

...

/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_4:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was 

9100/9100 [==============================] - 8s 825us/step - loss: 0.1024 - val_loss: 0.1208
Epoch 7/200
9100/9100 [==============================] - 7s 810us/step - loss: 0.1018 - val_loss: 0.1155
Epoch 8/200
 160/9100 [..............................] - ETA: 7s - loss: 0.0950

/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_4:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/Users/maojuntao/anaconda3/lib/python2.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was 

9100/9100 [==============================] - 7s 823us/step - loss: 0.1008 - val_loss: 0.1170
Epoch 9/200
9100/9100 [==============================] - 7s 802us/step - loss: 0.1003 - val_loss: 0.1193
Epoch 10/200
9100/9100 [==============================] - 7s 819us/step - loss: 0.0994 - val_loss: 0.1183
Epoch 11/200
9100/9100 [==============================] - 8s 832us/step - loss: 0.0969 - val_loss: 0.1181
Epoch 12/200
9100/9100 [==============================] - 7s 806us/step - loss: 0.0962 - val_loss: 0.1183
Epoch 13/200
9100/9100 [==============================] - 8s 832us/step - loss: 0.0959 - val_loss: 0.1189
Epoch 14/200
9100/9100 [==============================] - 8s 831us/step - loss: 0.0951 - val_loss: 0.1180
Epoch 15/200
9100/9100 [==============================] - 7s 813us/step - loss: 0.0950 - val_loss: 0.1182
Epoch 16/200
9100/9100 [==============================] - 7s 806us/step - loss: 0.0949 - val_loss: 0.1178
Epoch 17/200
9100/9100 [==============================] - 7s

"\n##################################################################################################################\n\n    ### ==============Below is concating with others' future branch convLSTM===========================\n    ## utility layers\n    flatten_layer = Flatten()\n    expand_dim_layer = Lambda(lambda x: K.expand_dims(x,1))\n    Concatenatelayer = Concatenate(axis=1)\n    get_dim1_layer = Lambda(lambda x: x[:,0,:])\n\n\n    ### ====================Graph def====================\n    # The first part is unchanged\n    encoder_inputs = Input(shape=(None, num_encoder_tokens))\n    encoder = LSTM(latent_dim, return_state=True)\n    encoder_outputs, state_h, state_c = encoder(encoder_inputs)\n    states = [state_h, state_c]\n\n\n    ###======convLSTM on others' future======\n    other_fut_lstm = ConvLSTM2D(filters=latent_dim, kernel_size=(num_user-1, 3),\n                       input_shape=(1, num_user-1, fps, 3),\n                       padding='same', return_sequences=True, r

In [4]:
print(_video_db.shape)

(2593, 10, 90)


In [13]:

print(input_seq.shape)

(1, 10, 90)


In [14]:
print(decoded_sentence.shape)

(1, 10, 6)


In [20]:
print(decoder_target.shape)

(1, 10, 6)


In [78]:
#print(decoder_target)
print('单次输出',decoder_outputs.shape)

NameError: name 'decoder_outputs' is not defined

In [21]:
video_data_train1 = pickle.load(open('/Users/maojuntao/Downloads/project/data 2/shanghai_dataset_xyz_train.p','rb')) 
video_data_train1 = clip_xyz(video_data_train1)
datadb1 = video_data_train1.copy()


_video_db1,_video_db_future1,_video_db_future_input1 = get_data(datadb1,pick_user=False)
    
num_testing_sample = 1
if cfg.input_mean_var:
    encoder_input_data1 = get_gt_target_xyz(_video_db1[:-num_testing_sample,:,:])
else:
    encoder_input_data1 = _video_db1[:-num_testing_sample,:,:]
decoder_target_data1 = get_gt_target_xyz(_video_db_future1)[:-num_testing_sample,:,:]
# decoder_input_data = get_gt_target_xyz(_video_db_future_input)[:-num_testing_sample,-1,:][:,np.newaxis,:]
decoder_input_data1 = get_gt_target_xyz(_video_db_future_input1)[:-num_testing_sample,0,:][:,np.newaxis,:]

('_video_ind = ', '098', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '099', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '214', 'per_video_db.shape = ', (28, 10, 90))
('_video_ind = ', '215', 'per_video_db.shape = ', (58, 10, 90))
('_video_ind = ', '212', 'per_video_db.shape = ', (84, 10, 90))
('_video_ind = ', '213', 'per_video_db.shape = ', (29, 10, 90))
('_video_ind = ', '210', 'per_video_db.shape = ', (28, 10, 90))
('_video_ind = ', '211', 'per_video_db.shape = ', (29, 10, 90))
('_video_ind = ', '090', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '092', 'per_video_db.shape = ', (102, 10, 90))
('_video_ind = ', '093', 'per_video_db.shape = ', (68, 10, 90))
video 094 only has 17 seconds. skip...
('_video_ind = ', '095', 'per_video_db.shape = ', (68, 10, 90))
('_video_ind = ', '096', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '097', 'per_video_db.shape = ', (136, 10, 90))
('_video_ind = ', '133', 'per_video_db.shape = ', (34, 10, 90))

In [31]:
print(encoder_inputs.shape)

NameError: name 'encoder_inputs' is not defined

In [24]:
print(encoder_input_data.shape)

(10112, 10, 90)


In [108]:
print(decoder_input_data.shape)

(10112, 1, 6)


NameError: name 'all_outputs' is not defined

In [27]:
print(input.shape)
print(_video_db.shape)
print(_video_db_future_input.shape)

(2593, 10, 90)
(2593, 10, 90)
(2593, 10, 90)


In [28]:
pickle.dump(_video_db,open('video_db.p','wb'))
pickle.dump(_video_db_future,open('video_db_future.p','wb'))
pickle.dump(_video_db_future_input,open('video_db_future_input.p','wb'))

In [37]:
if not cfg.input_mean_var:
        encoder_inputs = Input(shape=(None, num_encoder_tokens))
        print(1)
else:
    print(0)

1


In [38]:
print(decoder_target_data.shape)

(10112, 10, 6)


In [87]:
a = np.ones((2,10,90))
b = np.ones((2,300,3))
b[0,:,0] = a[0,:,:30].reshape(300)
b[0,:,1] = a[0,:,30:60].reshape(300)
b[0,:,2] = a[0,:,60:90].reshape(300)


In [43]:
b.shape

(2, 300, 3)

In [44]:
for i in range(10000):
    

(10, 30)

In [312]:
def get_unet(pretrained_weights = None):
    inputs = Input(shape=(300,3))
    conv1 = Conv1D(16, 3, activation='relu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv1D(16, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling1D(2,data_format='channels_last')(conv1)
    #
    conv2 = Conv1D(32, 3, activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv1D(32, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling1D(2,data_format='channels_last')(conv2)
    #
    conv3 = Conv1D(64, 3, activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv1D(64, 3, activation='relu', padding='same')(conv3)

    up1 = UpSampling1D(size=2 )(conv3)
    up1 = concatenate([conv2,up1],axis=2)
    conv4 = Conv1D(32, 3, activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv1D(32, 3, activation='relu', padding='same')(conv4)
    #
    up2 = UpSampling1D(size=2)(conv4)
    up2 = concatenate([conv1,up2], axis=2)
    conv5 = Conv1D(16,3, activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv1D(16, 3, activation='relu', padding='same')(conv5)
    #
    conv6 = Conv1D(6, 1, activation='relu',padding='same')(conv5)
    pool3 = MaxPooling1D(30,data_format='channels_last')(conv6)
    #conv6 = core.Reshape((6,patch_height*patch_width))(conv6)
    #conv6 = core.Permute((2,1))(conv6)
    ############
    print('pool3',pool3.shape)
    #decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(conv6)
    b0,b1,b2,b3,b4,b5,b6,b7,b8,b9=tf.split(conv6,10,1)
    print('b0',b0.shape)
    b0=tf.reduce_mean(b0, axis=1,keep_dims=True)
    b1=tf.reduce_mean(b1, axis=1,keep_dims=True)
    b2=tf.reduce_mean(b2, axis=1,keep_dims=True)
    b3=tf.reduce_mean(b3, axis=1,keep_dims=True)
    b4=tf.reduce_mean(b4, axis=1,keep_dims=True)
    b5=tf.reduce_mean(b5, axis=1,keep_dims=True)
    b6=tf.reduce_mean(b6, axis=1,keep_dims=True)
    b7=tf.reduce_mean(b7, axis=1,keep_dims=True)
    b8=tf.reduce_mean(b8, axis=1,keep_dims=True)
    b9=tf.reduce_mean(b9, axis=1,keep_dims=True)
    
    output=[]
    output.append(b0)
    output.append(b1)
    output.append(b2)
    output.append(b3)
    output.append(b4)
    output.append(b5)
    output.append(b6)
    output.append(b7)
    output.append(b8)
    output.append(b9)
    '''
    print('b0',b0.shape)
    c=tf.stack([b0,b1,b2,b3,b4,b5,b6,b7,b8,b9],1)
    print('tf.stack',c.shape)
    '''
    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(output)
    print(('1',encoder_inputs.shape))
    print(('2',decoder_outputs.shape))
    model = Model([inputs], pool3)

    # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.3, nesterov=False)


    return model

In [313]:
import numpy as np  
import os 
import skimage.io as io 
import skimage.transform as trans 
import numpy as np 
from keras.models import * 
from keras.layers import * 
from keras.optimizers import * 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler 
from keras import backend as keras

In [323]:
"""
-first part:
seq2seq without teacher forcing
-second part:
seq2seq without teacher forcing, with others' future convLSTM
concat states with decoder LSTM and then predict
"""
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers import Lambda,Concatenate,Flatten,ConvLSTM2D
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import backend as K
from keras.models import load_model
import sys,glob,io,random
if '/Users/maojuntao/Downloads/project/' not in sys.path:
    sys.path.insert(0, '/Users/maojuntao/Downloads/project/')
from dataLayer import DataLayer
import cost as costfunc
from config import cfg
from dataIO import clip_xyz
from utility import reshape2second_stacks,get_data
from utility import get_shuffle_index,shuffle_data,get_gt_target_xyz,get_gt_target_xyz_oth
from utility import slice_layer
from random import shuffle
import matplotlib.pyplot as plt
import cPickle as pickle
import numpy as np
import pdb


if __name__ == '__main__':
    #experiment = 1
    batch_size = 32  # Batch size for training.
    epochs = 200  # Number of epochs to train for.
    latent_dim = 64  # Latent dimensionality of the encoding space.

    fps = 30
    num_encoder_tokens = 3*fps
    num_decoder_tokens = 6
    max_encoder_seq_length = cfg.running_length
    max_decoder_seq_length = cfg.predict_step
    # num_user = 48


    model = get_unet()
    model.compile(optimizer='Adam', loss='mean_squared_error')
    #model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])


    #### ====================data====================
    ## load data just as in Fov_seq2seq.py
    # if cfg.use_xyz:
    #     all_video_data = pickle.load(open('/scratch/cl2840/360video/data/new_exp_'+str(experiment)+'_xyz.p','rb'))
    #     data_dim = 3
    # all_video_data = clip_xyz(all_video_data)
    # datadb = all_video_data.copy()
    ##### data format 3or4--4
    # video_data_train = pickle.load(open('/scratch/cl2840/360video/data/shanghai_dataset_xyz_train.p','rb'))    
    #### format5
    video_data_train = pickle.load(open('/Users/maojuntao/Downloads/project/data 2/shanghai_dataset_xyz_train.p','rb')) 
    video_data_train = clip_xyz(video_data_train)
    datadb = video_data_train.copy()


    _video_db,_video_db_future,_video_db_future_input = get_data(datadb,pick_user=False)
    # total_num_samples = _video_db.shape[0]

    if cfg.shuffle_data:
        #shuffle the whole dataset
        # index_shuf = get_shuffle_index(total_num_samples)
        index_shuf = pickle.load(open('index_shuf'+'_exp'+str(experiment)+'.p','rb'))
        _video_db = shuffle_data(index_shuf,_video_db)
        _video_db_future = shuffle_data(index_shuf,_video_db_future)
        _video_db_future_input = shuffle_data(index_shuf,_video_db_future_input)


    # num_testing_sample = int(0.15*total_num_samples)#use last few as test
    num_testing_sample = 1
    if cfg.input_mean_var:
        encoder_input_data = get_gt_target_xyz(_video_db[:-num_testing_sample,:,:])
    else:
        encoder_input_data = _video_db[:-num_testing_sample,:,:]
    decoder_target_data = get_gt_target_xyz(_video_db_future)[:-num_testing_sample,:,:]
    # decoder_input_data = get_gt_target_xyz(_video_db_future_input)[:-num_testing_sample,-1,:][:,np.newaxis,:]
    decoder_input_data = get_gt_target_xyz(_video_db_future_input)[:-num_testing_sample,0,:][:,np.newaxis,:]



    ### ====================Training====================
    # model = load_model('fov_s2s_noteacherforcing_epoch21-0.0759.h5')
    # tag = 'notFor_tanh_newdata_exp2_epoch'
    # tag = 'fctar_seqseq_shanghai_traintest_split_predmeanvar_Aug9'
    # tag = 'fctar_seqseq_shanghai_traintest_split_meanvarmeanvar_Aug9'
    tag = 'fctar_seqseq_THU_predmeanvar_Sep5'
    model_checkpoint = ModelCheckpoint(tag+'{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss', save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                     patience=3, min_lr=1e-6)
    stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
    b = np.ones((10112,300,3))
    for i in range(10112):
        b[i,:,0] = encoder_input_data[i,:,:30].reshape(300)
        b[i,:,1] = encoder_input_data[i,:,30:60].reshape(300)
        b[i,:,2] = encoder_input_data[i,:,60:90].reshape(300)
    print(b.shape)

    model.fit(b, decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.1,
              shuffle=True,initial_epoch=0,
              callbacks=[model_checkpoint, reduce_lr, stopping])



    ### ====================Testing====================
    


pool3 (?, 10, 6)
b0 (?, 30, 6)
('1', TensorShape([Dimension(None), Dimension(None), Dimension(90)]))
('2', TensorShape([Dimension(None), Dimension(10), Dimension(6)]))
('_video_ind = ', '098', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '099', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '214', 'per_video_db.shape = ', (28, 10, 90))
('_video_ind = ', '215', 'per_video_db.shape = ', (58, 10, 90))
('_video_ind = ', '212', 'per_video_db.shape = ', (84, 10, 90))
('_video_ind = ', '213', 'per_video_db.shape = ', (29, 10, 90))
('_video_ind = ', '210', 'per_video_db.shape = ', (28, 10, 90))
('_video_ind = ', '211', 'per_video_db.shape = ', (29, 10, 90))
('_video_ind = ', '090', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '092', 'per_video_db.shape = ', (102, 10, 90))
('_video_ind = ', '093', 'per_video_db.shape = ', (68, 10, 90))
video 094 only has 17 seconds. skip...
('_video_ind = ', '095', 'per_video_db.shape = ', (68, 10, 90))
('_video_ind = ', '096',

In [332]:
 ##### data format 3or4--4
    # video_data_test = pickle.load(open('/scratch/cl2840/360video/data/shanghai_dataset_xyz_test.p','rb'))
    ### data format 5
video_data_test = pickle.load(open('/Users/maojuntao/Downloads/project/data 2/shanghai_dataset_xyz_test.p','rb'))

video_data_test = clip_xyz(video_data_test)
datadb = video_data_test.copy()   ##arrange the test data
_video_db,_video_db_future,_video_db_future_input = get_data(datadb,pick_user=False)

if cfg.input_mean_var:
    video_db = get_gt_target_xyz(_video_db)

        # model = load_model('fov_s2s_noteacherforcing.h5')
        # model = load_model('fov_s2s_noteacherforcing_epoch74-0.0087.h5')
        # model = load_model('notFor_tanh_newdata_epoch18-0.0761.h5')
        # model = load_model('notFor_tanh_newdata_exp2_epoch200-0.0302.h5')
c=np.ones((2593,300,3))
def decode_sequence_fov(input_seq):
            # Encode the input as state vectors.
    '''
    last_location = input_seq[0,-1,:][np.newaxis,np.newaxis,:]
    if not cfg.input_mean_var:
        last_mu_var = get_gt_target_xyz(last_location)
    last_mu_var = last_location
    else:
    '''

    

    decoded_sentence = model.predict(input_seq)
    return decoded_sentence


gt_sentence_list = []
decoded_sentence_list = []
print('111',_video_db.shape)
for i in range(_video_db.shape[0]):
    c[i,:,0] = _video_db[i,:,:30].reshape(300)
    c[i,:,1] = _video_db[i,:,30:60].reshape(300)
    c[i,:,2] = _video_db[i,:,60:90].reshape(300)
        # for seq_index in range(total_num_samples-num_testing_sample,total_num_samples):
for seq_index in range(c.shape[0]):
            # Take one sequence (part of the training set)
            # for trying out decoding.
    input_seq = c[seq_index: seq_index + 1,:,:]

    decoded_sentence = decode_sequence_fov(input_seq)
    decoded_sentence_list+=[decoded_sentence]
    gt_sentence = _video_db_future[seq_index: seq_index + 1,:,:]
    gt_sentence_list+=[gt_sentence]
    decoder_target = get_gt_target_xyz(gt_sentence)
            # print('-')
            # print('Decoded sentence - decoder_target:', np.squeeze(np.array(decoded_sentence))[:,:3]-np.squeeze(decoder_target)[:,:3])

pickle.dump(decoded_sentence_list,open('decoded_sentence'+tag+'.p','wb'))
pickle.dump(gt_sentence_list,open('gt_sentence_list'+tag+'.p','wb'))
print('Testing finished!')

('_video_ind = ', '146', 'per_video_db.shape = ', (96, 10, 90))
('_video_ind = ', '153', 'per_video_db.shape = ', (96, 10, 90))
video 155 only has 19 seconds. skip...
('_video_ind = ', '154', 'per_video_db.shape = ', (32, 10, 90))
('_video_ind = ', '158', 'per_video_db.shape = ', (96, 10, 90))
('_video_ind = ', '190', 'per_video_db.shape = ', (93, 10, 90))
('_video_ind = ', '079', 'per_video_db.shape = ', (34, 10, 90))
('_video_ind = ', '128', 'per_video_db.shape = ', (136, 10, 90))
('_video_ind = ', '058', 'per_video_db.shape = ', (30, 10, 90))
('_video_ind = ', '136', 'per_video_db.shape = ', (68, 10, 90))
('_video_ind = ', '032', 'per_video_db.shape = ', (99, 10, 90))
('_video_ind = ', '019', 'per_video_db.shape = ', (33, 10, 90))
('_video_ind = ', '031', 'per_video_db.shape = ', (33, 10, 90))
('_video_ind = ', '118', 'per_video_db.shape = ', (68, 10, 90))
('_video_ind = ', '170', 'per_video_db.shape = ', (32, 10, 90))
('_video_ind = ', '160', 'per_video_db.shape = ', (64, 10, 90))
